<a href="https://colab.research.google.com/github/mihirsh73/Gender-Recognition/blob/master/gender_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [0]:
dff = pd.read_csv("Female-Names.csv")

In [0]:
dfm = pd.read_csv("Male-Names.csv")

In [0]:
df = pd.concat([dff,dfm])

In [0]:
df.head(-10)

,name,gender,race
0,shivani,f,indian
1,isha,f,indian
2,smt shyani devi,f,indian
3,divya,f,indian
4,mansi,f,indian
...,...,...,...
14830,pradeep kumar,m,indian
14831,ajeet singj,m,indian
14832,rajesh kumar,m,indian
14833,ashok shrivastav,m,indian


In [0]:
char2idx = {}
char_set = [' ', '.', 'END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
index = 0
for ch in char_set:
  char2idx[ch] = index
  index += 1

In [0]:
def should_keep(word):
  if len(word) > 19:
    return False
  for ch in word:
    if ch not in set(char_set):
      return False
  return True

In [0]:
def clean(word):

  char_set = [' ', '.', 'END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']   
  word = str(word)
  name = word.lower()
  if should_keep(name):
    return name
  else:
    return None

In [0]:
df.name = df.name.apply(lambda x : clean(x))
df = df.dropna()

In [0]:
df.head()

,name,gender,race
0,shivani,f,indian
1,isha,f,indian
2,smt shyani devi,f,indian
3,divya,f,indian
4,mansi,f,indian


In [0]:
max_word_len = 20
def convert_one_hot(word):
  word_chars =  []
  for ch in word:
    one_hot_vec = np.zeros(len(char_set), )
    one_hot_vec[char2idx[ch]] =1
    word_chars.append(one_hot_vec)
    
    
  for _ in range(max_word_len- len(word)):
    one_hot_vec = np.zeros(len(char_set),)
    one_hot_vec[char2idx["END"]] = 1
    word_chars.append(one_hot_vec)
  word_chars = np.array(word_chars)
  return word_chars
   

In [0]:
words = []
labels = []
for n,g in df.drop("race", axis = 1). itertuples(index = False):
  words.append(convert_one_hot(n))
  labels.append(g)

In [0]:
words = np.array(words)
labels = np.array(labels)

In [0]:
words.shape

(27980, 20, 29)

In [0]:
ohe = OneHotEncoder()
y = ohe.fit_transform(labels.reshape(-1,1)).todense()

In [0]:
y[:5]

matrix([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])

In [0]:
model = Sequential()
model.add(LSTM(64,input_shape = (20,29), return_sequences = True))
model.add(LSTM(64))
model.add(Dropout (.3))
model.add(Dense(2, activation = 'sigmoid'))
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20, 64)            24064     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 57,218
Trainable params: 57,218
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop' , loss = 'binary_crossentropy' , metrics = ['acc'])




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
X_train, X_test, y_train, y_test = train_test_split(words, y, test_size=0.33, random_state=42)

In [0]:
model.fit(X_train , y_train , epochs = 5)




Epoch 1/5





18746/18746 [==============================] - 20s 1ms/step - loss: 0.5322 - acc: 0.7345
Epoch 2/5
18746/18746 [==============================] - 19s 1ms/step - loss: 0.3991 - acc: 0.8293
Epoch 3/5
18746/18746 [==============================] - 19s 1ms/step - loss: 0.3440 - acc: 0.8607
Epoch 4/5
18746/18746 [==============================] - 19s 1ms/step - loss: 0.3077 - acc: 0.8778
Epoch 5/5
18746/18746 [==============================] - 18s 986us/step - loss: 0.2777 - acc: 0.8939


In [0]:
model.evaluate(X_test, y_test)

9234/9234 [==============================] - 2s 203us/step


[0.27993428209449195, 0.892354342595101]

In [0]:
name = "juhi"
model.predict(convert_one_hot(name).reshape(1 , 20 , 29))

array([[0.95673   , 0.04597969]], dtype=float32)